# Random Forest & XGBoost — training + cross-validation

This notebook trains supervised classifiers to predict whether a `(polish_label, name_variant)` pair is a true match, using precomputed string/phonetic metric features.

---

## Imports

In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from skopt.space import Real, Integer, Categorical

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score, accuracy_score, roc_curve, auc

from skopt import BayesSearchCV
from skopt.space import Integer, Categorical

from xgboost import XGBClassifier

---

## Global configuration

In [ ]:
warnings.filterwarnings("ignore")

RANDOM_SEED = 42

# Number of samples from each class (balanced dataset size = 2*N)
N_SAMPLES_PER_CLASS = 500

# Number of folds in stratified K-fold CV
N_FOLDS = 5

# Decision thresholds applied to predicted probabilities
THRESHOLDS = [0.35, 0.40, 0.45, 0.50]



In [ ]:
# df_merged = pd.read_parquet("merged_weigths.parquet")
df_gold = pd.read_parquet("wd_dataset.parquet")
df_gold["merge_col"] = df_gold.apply(lambda row: tuple([row.polish_label, row.name_variant]), axis=1)

print("df_gold:", df_gold.shape)
df_merged= pd.read_parquet("df_merged.parquet")

---

##  Define X (features) and y (target label)

In [ ]:

y_full = df_merged["truth"].astype(int).values

# Feature columns: renamed metric columns (coverages, and lengths of strings list)
feature_cols = [c for c in df_merged.columns if c.startswith(("coverage", "len"))]
X_full = df_merged[feature_cols].values

print("Number of features:", len(feature_cols))
print("Feature names:", feature_cols)
print("Class distribution [0, 1]:", np.bincount(y_full))

---

## Create balanced training subset

Sample equal numbers of positives and negatives.

In [ ]:
rng = np.random.default_rng(RANDOM_SEED)

pos_idx = np.where(y_full == 1)[0]
neg_idx = np.where(y_full == 0)[0]

n = min(N_SAMPLES_PER_CLASS, len(pos_idx), len(neg_idx))

sample_idx = np.concatenate([
    rng.choice(pos_idx, size=n, replace=False),
    rng.choice(neg_idx, size=n, replace=False),
])
rng.shuffle(sample_idx)

X_sampled = X_full[sample_idx]
y_sampled = y_full[sample_idx]

print("Sampled shape:", X_sampled.shape)
print("Sampled class counts [0, 1]:", np.bincount(y_sampled))
print("Positive ratio:", np.mean(y_sampled))

---

## Random Forest: Bayesian search space



In [ ]:
rf_search_space = {
    "n_estimators": Integer(100, 1000),
    "max_depth": Integer(5, 30),
    "min_samples_split": Integer(2, 20),
    "min_samples_leaf": Integer(1, 10),
    "max_features": Categorical(["sqrt", "log2"]),
}

print("RF search space defined")

---

## Random Forest: Stratified K-Fold + threshold sweep (main loop)


In [ ]:
search_spaces_rf = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Categorical(['sqrt', 'log2'])
}

In [ ]:
print("=" * 60)
print("RANDOM FOREST CLASSIFIER")
print("=" * 60)
print(f"Starting {N_FOLDS}-fold cross-validation...")
print(f"Testing thresholds: {THRESHOLDS}")
print(f"Sample size: {len(y_sampled)} | Train per fold: {int(len(y_sampled) * (N_FOLDS - 1) / N_FOLDS)}")

# Model definition (here you can easily swap RandomForest for XGBoost, etc.)

# Stratified K-Fold ensures the class proportion is preserved in each fold
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

beta = 2

# Containers for results
results_by_threshold = {t: {
    "val_f2": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f2": []
} for t in THRESHOLDS}

best_params_history = []

print(f"RandomForestClassifier: Starting {N_FOLDS}-fold cross-validation...")
print(f"Thresholds being tested: {THRESHOLDS}")
print(f"Sample size: {len(y_sampled)} (Training per fold: ~{int(len(y_sampled) * 0.8)})")

fold_no = 1
for train_idx_local, test_idx_local in skf.split(X_sampled, y_sampled):
    # 1) Prepare the datasets
    X_train_fold = X_sampled[train_idx_local]
    y_train_fold = y_sampled[train_idx_local]
    X_test_fold = X_sampled[test_idx_local]
    y_test_fold = y_sampled[test_idx_local]

    # Masks and the global set
    global_train_idx = sample_idx[train_idx_local]
    mask_train = np.zeros(len(df_merged), dtype=bool)
    mask_train[global_train_idx] = True
    X_test_global = X_full[~mask_train]

    # 2) Bayesian hyperparameter optimization
    opt = BayesSearchCV(
        estimator=RandomForestClassifier(n_jobs=-1, random_state=RANDOM_SEED),
        search_spaces=search_spaces_rf,
        scoring='f1',  # We optimize F1 (harmonic mean), because F2 isn't available as a standard string scorer
        cv=3,
        n_iter=15,  # Reduced a bit for speed; you can set 20
        n_jobs=-1,
        random_state=RANDOM_SEED,
        verbose=0
    )
    opt.fit(X_train_fold, y_train_fold)
    best_model_rf = opt.best_estimator_
    best_params_rf = dict(opt.best_params_)
    best_params_history.append(best_params_rf)
    print(f"\nFold {fold_no}: Params: {best_params_rf}")

    # Compute probabilities ONCE for this fold/model
    probs_local = best_model_rf.predict_proba(X_test_fold)[:, 1]
    probs_global = best_model_rf.predict_proba(X_test_global)[:, 1]

    # Prepare Ground Truth for filtering (shared within the fold)
    train_positive_indices = global_train_idx[y_train_fold == 1]
    train_positive_merge_cols = df_merged.iloc[train_positive_indices]["merge_col"]
    df_res_filtered = df_gold[~df_gold.merge_col.isin(train_positive_merge_cols)]

    # 3) Loop over thresholds (evaluate many variants at once)
    best_fold_stats = {"f2_global": -1, "thr": 0, "p": 0, "r": 0, "val_f2": 0}
    for t in THRESHOLDS:
        # --- Local evaluation (Validation) ---
        preds_local_t = (probs_local >= t).astype(int)
        val_f2 = fbeta_score(y_test_fold, preds_local_t, beta=2, zero_division=0)
        val_acc = accuracy_score(y_test_fold, preds_local_t)

        # --- Global evaluation ("Rest of the world") ---
        val_preds_global = (probs_global >= t).astype(int)

        # Business logic (Set-based metrics)
        df_test_fold = df_merged[~mask_train].copy()
        df_test_fold["result"] = val_preds_global

        res_filt = df_test_fold[df_test_fold["result"] == 1]
        org_set = set(df_res_filtered[["polish_label", "name_variant"]].itertuples(index=False, name=None))
        new_set = set(res_filt[["polish_label", "name_variant"]].itertuples(index=False, name=None))

        true_positive_count = len(new_set & org_set)
        false_positive_count = len(new_set - org_set)
        false_negative_df = df_res_filtered.loc[~df_res_filtered["merge_col"].isin(res_filt["merge_col"])]
        false_negative_count = len(false_negative_df)

        prec = true_positive_count / (true_positive_count + false_positive_count) if (
            true_positive_count + false_positive_count) > 0 else 0
        rec = true_positive_count / (true_positive_count + false_negative_count) if (
            true_positive_count + false_negative_count) > 0 else 0
        f2_global = (1 + beta ** 2) * (prec * rec) / (beta ** 2 * prec + rec) if (beta ** 2 * prec + rec) > 0 else 0

        # Store results for this threshold
        results_by_threshold[t]["val_f2"].append(val_f2)
        results_by_threshold[t]["accuracy"].append(val_acc)
        results_by_threshold[t]["precision"].append(prec)
        results_by_threshold[t]["recall"].append(rec)
        results_by_threshold[t]["f2"].append(f2_global)

        # --- Update the best result within this fold ---
        if f2_global > best_fold_stats["f2_global"]:
            best_fold_stats = {
                "f2_global": f2_global,
                "thr": t,
                "p": prec,
                "r": rec,
                "val_f2": val_f2
            }

    # Print per fold
    print(
        f"Fold {fold_no}: Best Thr: {best_fold_stats['thr']:.2f} | Val F2: {best_fold_stats['val_f2']:.4f} | Global F2: {best_fold_stats['f2_global']:.4f} (P: {best_fold_stats['p']:.2f}, R: {best_fold_stats['r']:.2f})"
    )
    fold_no += 1


---

##  Results

In [ ]:
# --- SUMMARY OF PARAMETERS (Mode / Most Frequent) ---
params_df = pd.DataFrame(best_params_history)
print("\n" + "=" * 50)
print("HYPERPARAMETER ANALYSIS (Best Params from 5 folds)")
print("=" * 50)

# We compute the most frequent value (mode) for each parameter
most_common_params = params_df.mode().iloc[0].to_dict()
print("Most frequent configuration (Mode):")
for k, v in most_common_params.items():
    # If parameters are numbers, format nicely; if strings, leave as-is
    val_str = f"{int(v)}" if isinstance(v, (int, np.integer)) else f"{v}"
    print(f"  {k}: {val_str}")


# --- SUMMARY OF RESULTS FOR EACH THRESHOLD ---
# 1) First, we find the winning threshold (highest mean F2)
best_avg_f2 = -1
best_t_final = None

for t in THRESHOLDS:
    current_avg_f2 = np.mean(results_by_threshold[t]['f2'])
    if current_avg_f2 > best_avg_f2:
        best_avg_f2 = current_avg_f2
        best_t_final = t


# 2) Display results, marking the winner in the header
for t in THRESHOLDS:
    res = results_by_threshold[t]

    # Check whether this is the winner
    if t == best_t_final:
        header_suffix = " 🏆 "
    else:
        header_suffix = ""

    print(f"\n=== {header_suffix} FINAL RESULTS (Mean +/- Std) for Threshold: {t:.2f} ===")
    print(f"Val F2 (Local): {np.mean(res['val_f2']):.4f} ± {np.std(res['val_f2']):.4f}")
    print(f"Accuracy (Val): {np.mean(res['accuracy']):.4f} ± {np.std(res['accuracy']):.4f}")
    print("-" * 40)
    print(f"Precision:      {np.mean(res['precision']):.4f} ± {np.std(res['precision']):.4f}")
    print(f"Recall:         {np.mean(res['recall']):.4f} ± {np.std(res['recall']):.4f}")
    print(f"F2-Score:       {np.mean(res['f2']):.4f} ± {np.std(res['f2']):.4f}")


# precision: 0.6469689849624061
# recall: 0.7762898223851141
# F2: 0.7464487096074605
# threshold: 0.4


### Plot

In [ ]:
# 1. Convert the nested dictionary `results_by_threshold` into a flat DataFrame
data_rows = []

for t, metrics in results_by_threshold.items():
    # `metrics` is a dictionary of lists, e.g. [0.7, 0.72, ...] for each fold
    n_folds_actual = len(metrics['f2'])  # should be 5

    for i in range(n_folds_actual):
        # Add rows for each metric we care about
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Global F2', 'Score': metrics['f2'][i]})
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Precision', 'Score': metrics['precision'][i]})
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Recall', 'Score': metrics['recall'][i]})
        # Optionally add Val F2 if you want to compare it
        # data_rows.append({'Threshold': t, 'Fold': i+1, 'Metric': 'Local Val F2', 'Score': metrics['val_f2'][i]})

df_metrics = pd.DataFrame(data_rows)



In [ ]:
results_by_threshold

In [ ]:
# 2. Style configuration
sns.set_theme(style="whitegrid", context="paper", font_scale=1.2)
plt.figure(figsize=(10, 7))  # Slightly taller chart to fit the legend at the bottom


# 3. Define colors (Manual for full control—matching your screenshot)
# Global F2 (Blue), Precision (Orange), Recall (Green)
colors = sns.color_palette("deep", 3)


# 4. Draw the plot
ax = sns.barplot(
    data=df_metrics,
    x="Threshold",
    y="Score",
    hue="Metric",
    palette=colors,  # Use the "deep" palette (as in the screenshot) instead of viridis
    capsize=.05,  # Slightly more subtle error bar caps
    errorbar="sd",
    edgecolor="0.3",  # Dark gray border for contrast
    linewidth=1
)


# 5. Titles and axes
plt.title('Impact of Threshold on Model Performance (5-Fold CV)', fontsize=16, weight='bold', y=1.02)
plt.xlabel('Decision Threshold', fontsize=13, labelpad=10)
plt.ylabel('Score (0-1)', fontsize=13, labelpad=10)
plt.ylim(0, 1.1)


# 6. IMPROVED LEGEND (Placed below the plot)
plt.legend(
    title=None,  # Remove the unnecessary "Metric" title
    loc='upper center',  # Legend anchor point
    bbox_to_anchor=(0.5, -0.15),  # Shift below the X axis
    ncol=3,  # All items on one line
    frameon=False,  # No border/frame
    fontsize=12
)


# 7. Values on bars (Annotation)
for container in ax.containers:
    # Get bar heights and format them
    labels = [f'{v.get_height():.2f}' for v in container]

    ax.bar_label(
        container,
        labels=labels,
        label_type='center',  # Text in the middle of the bar
        padding=0,
        fontsize=10,
        color='white',  # White text
        weight='bold'  # Bold for readability
    )


# Remove top and right spines
sns.despine()


# Important: adjust layout so the legend is not cut off when saving
plt.subplots_adjust(bottom=0.2)


# Save
output_path = "metrics_by_threshold__rf.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()


## XGB Classifier

In [ ]:
# XGBoost has different parameters than Random Forest.
# The key one is `scale_pos_weight`, which helps "force" higher Recall (important for F2).
search_spaces_xgb = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(3, 10),  # Boosting prefers shallower trees than RF
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'subsample': Real(0.6, 1.0),  # Prevents overfitting
    'colsample_bytree': Real(0.6, 1.0),  # Equivalent of max_features
    'scale_pos_weight': Integer(1, 6)  # MAGIC FOR F2: gives the positive class more weight
}


In [ ]:
print("=" * 60)
print("RANDOM FOREST CLASSIFIER")
print("=" * 60)
print(f"Starting {N_FOLDS}-fold cross-validation...")
print(f"Testing thresholds: {THRESHOLDS}")
print(f"Sample size: {len(y_sampled)} | Train per fold: {int(len(y_sampled) * (N_FOLDS - 1) / N_FOLDS)}")


# Stratified K-Fold zapewnia, że w każdym "foldzie" proporcja klas jest zachowana
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

# F2
beta = 2

# Listy na wyniki
results_by_threshold = {t: {
    "val_f2": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f2": []
} for t in THRESHOLDS}

best_params_history = []


fold_no = 1
for train_idx_local, test_idx_local in skf.split(X_sampled, y_sampled):
    # --- 1) Local train/test within the sampled subset
    X_train_fold = X_sampled[train_idx_local]
    y_train_fold = y_sampled[train_idx_local]
    X_test_fold = X_sampled[test_idx_local]
    y_test_fold = y_sampled[test_idx_local]

    # Global test pool = all rows NOT in fold train set
    global_train_idx = sample_idx[train_idx_local]
    mask_train = np.zeros(len(df_merged), dtype=bool)
    mask_train[global_train_idx] = True
    X_test_global = X_full[~mask_train]

    # --- 2) Bayesian hyperparameter search
    opt = BayesSearchCV(
        estimator=XGBClassifier(
            n_jobs=-1,
            random_state=RANDOM_SEED,
            eval_metric='logloss', 
            use_label_encoder=False
        ),
        search_spaces=rf_search_space,
        scoring='f1',
        cv=3,
        n_iter=15,
        n_jobs=-1,
        random_state=RANDOM_SEED,
        verbose=0
    )
    opt.fit(X_train_fold, y_train_fold)
    best_model_xgb = opt.best_estimator_
    best_params_xgb = dict(opt.best_params_)
    best_params_history.append(best_params_xgb)
    print(f"\nFold {fold_no}: Params: {best_params_xgb}")

    # --- 3) Predict probabilities once for both local and global test sets
    probs_local = best_model_xgb.predict_proba(X_test_fold)[:, 1]
    probs_global = best_model_xgb.predict_proba(X_test_global)[:, 1]

    # --- 4) Filter gold set to fold-specific positives (leakage control)
    train_positive_indices = global_train_idx[y_train_fold == 1]
    train_positive_merge_cols = df_merged.iloc[train_positive_indices]["merge_col"]
    df_res_filtered = df_gold[~df_gold.merge_col.isin(train_positive_merge_cols)]

    # --- 5) Loop over thresholds and evaluate
    best_fold_stats = {"f2_global": -1, "thr": 0, "p": 0, "r": 0, "val_f2": 0}
    for t in THRESHOLDS:
        # Local validation metrics
        preds_local_t = (probs_local >= t).astype(int)
        val_f2 = fbeta_score(y_test_fold, preds_local_t, beta=2, zero_division=0)
        val_acc = accuracy_score(y_test_fold, preds_local_t)

        # Global prediction pool
        val_preds_global = (probs_global >= t).astype(int)

        # Set-based metrics
        df_test_fold = df_merged[~mask_train].copy()
        df_test_fold["result"] = val_preds_global

        res_filt = df_test_fold[df_test_fold["result"] == 1]
        org_set = set(df_res_filtered[["polish_label", "name_variant"]].itertuples(index=False, name=None))
        new_set = set(res_filt[["polish_label", "name_variant"]].itertuples(index=False, name=None))

        true_positive_count = len(new_set & org_set)
        false_positive_count = len(new_set - org_set)
        false_negative_df = df_res_filtered.loc[~df_res_filtered["merge_col"].isin(res_filt["merge_col"])]
        false_negative_count = len(false_negative_df)

        prec = true_positive_count / (true_positive_count + false_positive_count) if (
                                                                                             true_positive_count + false_positive_count) > 0 else 0
        rec = true_positive_count / (true_positive_count + false_negative_count) if (
                                                                                            true_positive_count + false_negative_count) > 0 else 0
        f2_global = (1 + beta ** 2) * (prec * rec) / (beta ** 2 * prec + rec) if (beta ** 2 * prec + rec) > 0 else 0

        # Store results for this threshold across folds
        results_by_threshold[t]["val_f2"].append(val_f2)
        results_by_threshold[t]["accuracy"].append(val_acc)
        results_by_threshold[t]["precision"].append(prec)
        results_by_threshold[t]["recall"].append(rec)
        results_by_threshold[t]["f2"].append(f2_global)

        # Update best threshold for this fold
        if f2_global > best_fold_stats["f2_global"]:
            best_fold_stats = {
                "f2_global": f2_global,
                "thr": t,
                "p": prec,
                "r": rec,
                "val_f2": val_f2
            }

    # Print per fold
    print(
        f"Fold {fold_no}: Best Thr: {best_fold_stats['thr']:.2f} | Val F2: {best_fold_stats['val_f2']:.4f} | Global F2: {best_fold_stats['f2_global']:.4f} (P: {best_fold_stats['p']:.2f}, R: {best_fold_stats['r']:.2f})")
    fold_no += 1


### Results

In [ ]:
# --- SUMMARY OF PARAMETERS (Mode / Most Frequent) ---
params_df = pd.DataFrame(best_params_history)
print("\n" + "=" * 50)
print("HYPERPARAMETER ANALYSIS (Best Params from 5 folds)")
print("=" * 50)

# Compute the most frequent value (mode) for each parameter
most_common_params = params_df.mode().iloc[0].to_dict()
print("Most frequent configuration (Mode):")
for k, v in most_common_params.items():
    # If parameters are numbers, format nicely; if strings, leave as-is
    val_str = f"{int(v)}" if isinstance(v, (int, np.integer)) else f"{v}"
    print(f"  {k}: {val_str}")


# --- SUMMARY OF RESULTS FOR EACH THRESHOLD ---
# 1) First, find the winning threshold (highest average F2)
best_avg_f2 = -1
best_t_final = None

for t in THRESHOLDS:
    current_avg_f2 = np.mean(results_by_threshold[t]['f2'])
    if current_avg_f2 > best_avg_f2:
        best_avg_f2 = current_avg_f2
        best_t_final = t


# 2) Print results, marking the winner in the header
for t in THRESHOLDS:
    res = results_by_threshold[t]

    # Check if this is the winner
    if t == best_t_final:
        header_suffix = " 🏆 "
    else:
        header_suffix = ""

    print(f"\n=== {header_suffix} FINAL RESULTS (Mean +/- Std) for Threshold: {t:.2f} ===")
    print(f"Val F2 (Local): {np.mean(res['val_f2']):.4f} ± {np.std(res['val_f2']):.4f}")
    print(f"Accuracy (Val): {np.mean(res['accuracy']):.4f} ± {np.std(res['accuracy']):.4f}")
    print("-" * 40)
    print(f"Precision:      {np.mean(res['precision']):.4f} ± {np.std(res['precision']):.4f}")
    print(f"Recall:         {np.mean(res['recall']):.4f} ± {np.std(res['recall']):.4f}")
    print(f"F2-Score:       {np.mean(res['f2']):.4f} ± {np.std(res['f2']):.4f}")




In [ ]:
# 1. Convert the nested dictionary `results_by_threshold` into a flat DataFrame
data_rows = []

for t, metrics in results_by_threshold.items():
    # `metrics` is a dictionary of lists, e.g. [0.7, 0.72, ...] for each fold
    n_folds_actual = len(metrics['f2'])  # should be 5

    for i in range(n_folds_actual):
        # Add rows for each metric we care about
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Global F2', 'Score': metrics['f2'][i]})
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Precision', 'Score': metrics['precision'][i]})
        data_rows.append({'Threshold': t, 'Fold': i + 1, 'Metric': 'Recall', 'Score': metrics['recall'][i]})
        # Optionally add Val F2 if you want to compare it
        # data_rows.append({'Threshold': t, 'Fold': i+1, 'Metric': 'Local Val F2', 'Score': metrics['val_f2'][i]})

df_metrics = pd.DataFrame(data_rows)

# Let's see what it looks like
# df_metrics


# 2. Style configuration
sns.set_theme(style="whitegrid", context="paper", font_scale=1.2)
plt.figure(figsize=(10, 7))  # Slightly taller chart to fit the legend at the bottom


# 3. Define colors (Manual for full control—matching your screenshot)
# Global F2 (Blue), Precision (Orange), Recall (Green)
colors = sns.color_palette("deep", 3)


# 4. Draw the plot
ax = sns.barplot(
    data=df_metrics,
    x="Threshold",
    y="Score",
    hue="Metric",
    palette=colors,  # Use the "deep" palette (as in the screenshot) instead of viridis
    capsize=.05,  # Slightly more subtle error bar caps
    errorbar="sd",
    edgecolor="0.3",  # Dark gray border for contrast
    linewidth=1
)


# 5. Titles and axes
plt.title('Impact of Threshold on Model Performance (5-Fold CV)', fontsize=16, weight='bold', y=1.02)
plt.xlabel('Decision Threshold', fontsize=13, labelpad=10)
plt.ylabel('Score (0-1)', fontsize=13, labelpad=10)
plt.ylim(0, 1.1)


# 6. IMPROVED LEGEND (Placed below the plot)
plt.legend(
    title=None,  # Remove the unnecessary "Metric" title
    loc='upper center',  # Legend anchor point
    bbox_to_anchor=(0.5, -0.15),  # Shift below the X axis
    ncol=3,  # All items on one line
    frameon=False,  # No border/frame
    fontsize=12
)


# 7. Values on bars (Annotation)
for container in ax.containers:
    # Get bar heights and format them
    labels = [f'{v.get_height():.2f}' for v in container]

    ax.bar_label(
        container,
        labels=labels,
        label_type='center',  # Text in the middle of the bar
        padding=0,
        fontsize=10,
        color='white',  # White text
        weight='bold'  # Bold for readability
    )


# Remove top and right spines
sns.despine()


# Important: adjust layout so the legend is not cut off when saving
plt.subplots_adjust(bottom=0.2)


# Save
output_path = "metrics_by_threshold__xgb.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()


# ROC

In [ ]:
from typing import Tuple

def plot_roc_curve(
        model: RandomForestClassifier,
        X: np.ndarray,
        y: np.ndarray,
        title: str = "ROC curve (Random Forest)",
        save_path: Path | None = None,
        mark_threshold: float | None = None,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    """
    Plot ROC curve for a binary classifier and optionally save to file.

    Returns fpr, tpr, thresholds, auc_value.
    """
    proba = model.predict_proba(X)[:, 1]
    fpr, tpr, thresholds = roc_curve(y, proba)
    auc_value = auc(fpr, tpr)

    fig, ax = plt.subplots(figsize=(4.0, 4.0), dpi=300)

    # ROC curve
    ax.plot(fpr, tpr, lw=1.8, label=f"AUC = {auc_value:.3f}")

    # Diagonal (random classifier)
    ax.plot([0.0, 1.0], [0.0, 1.0], "--", lw=1.0, label="Random")

    # Optional: mark chosen operating point (e.g. best_t_final = 0.45)
    if mark_threshold is not None:
        idx = np.argmin(np.abs(thresholds - mark_threshold))
        ax.scatter(
            fpr[idx],
            tpr[idx],
            s=25,
            marker="o",
            zorder=3,
            label=f"Threshold = {mark_threshold:.2f}",
        )

    ax.set_xlim(0.0, 1.0)
    ax.set_ylim(0.0, 1.02)
    ax.set_xlabel("False positive rate")
    ax.set_ylabel("True positive rate")
    ax.set_title(title)
    ax.set_aspect("equal", "box")
    ax.grid(True, linestyle=":", linewidth=0.5)
    ax.legend(loc="lower right", frameon=True)

    fig.tight_layout()

    if save_path is not None:
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, bbox_inches="tight")
        plt.close(fig)

    return fpr, tpr, thresholds, auc_value


# ROC RF

In [ ]:
fig_path =  Path("figures/roc_rf_similarity.pdf")

best_t_final = 0.45 # or whatever was the best threshold from the code

fpr, tpr, thresholds, auc_value = plot_roc_curve(
    model=best_model_rf,
    X=X_sampled,
    y=y_sampled,
    title="Random Forest - ROC curve",
    save_path=fig_path,
    mark_threshold=best_t_final,
)

print(f"AUC: {auc_value:.3f}")
print(f"Wykres zapisany do: {fig_path}")

# ROC XGB

In [ ]:
fig_path =  Path("figures/roc_xgb_similarity.pdf")

best_t_final = 0.45 # or whatever was the best threshold from the code

fpr, tpr, thresholds, auc_value = plot_roc_curve(
    model=best_model_xgb,
    X=X_sampled,
    y=y_sampled,
    title="XGBoost - ROC curve",
    save_path=fig_path,
    mark_threshold=best_t_final,
)

print(f"AUC: {auc_value:.3f}")
print(f"Wykres zapisany do: {fig_path}")